In [ ]:
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.utils.class_weight import compute_sample_weight

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [128]:
df = pd.read_csv("C:/Users/Настя/YandexDisk-n4skolesnikova/HSE 4th year/Graduation Thesis/ACCIDENT_LEVEL_DATA.csv")
df = df.drop('Unnamed: 0', axis=1)
print(df.shape)
df.head()

(440132, 73)


,date,COORD_L,COORD_W,dor_k,s_dtp,RTA_vehicle_number,RTA_participants_number,RTA_number,number_of_deaths,number_of_wounded,...,markings_or_signs,traffic_control_violation,signal_violation,speeding,technical_malfunction,lighting_cat,road_object_type_cat,severity,severity_detailed,is_toll
0,31.01.2015,81.151944,53.740000,5.0,610,1,3,161242174,0,3,...,0,0,0,0,0,4,12,medium,injury_with_hospitalization,0
1,30.01.2015,85.018056,51.684444,6.0,200,2,3,161105683,0,2,...,0,0,0,0,0,0,12,medium,injury_with_hospitalization,0
2,30.01.2015,81.250000,53.818056,5.0,210,2,3,161763431,0,1,...,0,0,0,0,0,0,12,light,injury_no_hospitalization,0
3,24.01.2015,51.000000,84.000000,7.0,610,1,2,160331994,0,1,...,0,0,0,1,0,2,12,light,injury_no_hospitalization,0
4,23.01.2015,84.000000,53.000000,7.0,600,1,2,160213415,1,1,...,0,0,0,0,0,2,12,severe,death_at_scene,0


In [129]:
df.columns

Index(['date', 'COORD_L', 'COORD_W', 'dor_k', 's_dtp', 'RTA_vehicle_number',
       'RTA_participants_number', 'RTA_number', 'number_of_deaths',
       'number_of_wounded', 'RTA_time', 'any_TV_technical_failures',
       'any_non_private_vehicles', 'any_russian_vehicles',
       'any_white_vehicles', 'any_black_vehicles', 'any_colored_vehicles',
       'any_drunk_drivers', 'any_female_drivers', 'any_escaped_participants',
       'any_concomitant_traffic_violations',
       'any_injuries_without_safety_belt', 'num_drunk_participants',
       'num_child_restraints', 'num_cyclists', 'num_pedestrians',
       'min_vehicles_age', 'max_vehicles_age', 'avg_vehicles_age',
       'num_A_class_vehicles', 'num_B_class_vehicles', 'num_C_class_vehicles',
       'num_D_class_vehicles', 'num_E_class_vehicles', 'num_S_class_vehicles',
       'num_non_car_vehicles', 'num_front_wheel_drive_vehicles',
       'num_rear_wheel_drive_vehicles', 'num_four_wheel_drive_vehicles',
       'num_guilty_objects', 's

In [130]:
print(f"Final check for the gaps: {df.isna().any().sum()} gaps")

Final check for the gaps: 0 gaps


#### Create DataFrame for testing hypotheses.

In [131]:
df['severity'].unique()

array(['medium', 'light', 'severe'], dtype=object)

In [132]:
# manually encode target

replace_map = {
    'light': '1',
    'medium': '2',
    'severe': '3'
}

df['severity_encode'] = df['severity'].replace(replace_map)
df['severity_encode'] = df['severity_encode'].astype(int)
df['severity_encode'].unique()

array([2, 1, 3])

In [133]:
df_hypoth = df.drop(columns=['severity_detailed', 'severity'])

# Econometrics

## Multinomial Logit (MNL) regression

Earlier, a class imbalance was found in the 'is_toll' variable: the positive class makes up less than 7% of the entire dataset. Therefore, the results of hypothesis testing on such data will not be representative.  
To address the class imbalance problem, we will try two methods:

1. Balancing using undersampling  
2. Balancing using class weighting

### 1. undersampling 

In [134]:
df_balanced = pd.concat([
    df_hypoth[df_hypoth['is_toll'] == 1],
    df_hypoth[df_hypoth['is_toll'] == 0].groupby('severity_encode').apply(
        lambda x: x.sample(n=(df_hypoth[df_hypoth['is_toll'] == 1]['severity_encode'] == x.name).sum(),
                          random_state=42)
    ).reset_index(drop=True)
])

Let's start the analysis by building a baseline model that includes the `'is_toll'` variable along with key factors related to the road environment and driver experience.

In [135]:
formula = 'severity_encode ~ is_toll + drivers_avg_experience + guilty_drivers_avg_experience\
                + lighting_cat + road_object_type_cat + dor_z_cat + factor_cat'

model = smf.mnlogit(formula, data=df_balanced)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 1.037139
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:        severity_encode   No. Observations:                59080
Model:                        MNLogit   Df Residuals:                    59064
Method:                           MLE   Df Model:                           14
Date:                Sun, 20 Apr 2025   Pseudo R-squ.:                0.006764
Time:                        06:04:53   Log-Likelihood:                -61274.
converged:                       True   LL-Null:                       -61691.
Covariance Type:            nonrobust   LLR p-value:                4.535e-169
            severity_encode=2       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -0.1101      0.098     -1.127      0.

The coefficient for `is_toll` in medium-severity crashes (severity_encode = 2) is 0.0649 and statistically significant (p < 0.001), indicating a higher likelihood of medium-severity crashes compared to minor ones on toll roads. However, for severe crashes (severity_encode = 3), the `is_toll` coefficient is negative (−0.0496) and borderline significant (p ≈ 0.073).  
Therefore, **Hypothesis 2.1 is partially supported**: medium-severity crashes are statistically more likely than minor ones on toll roads, but no significant increase is observed for severe outcomes.

The next step is to shift the focus from the external environment to behavioral and regulatory aspects of crashes, including traffic violations and the presence of drunk drivers. This will allow us to explore an alternative explanation for the observed differences.

In [136]:
formula = 'severity_encode ~ is_toll + speeding + signal_violation + wrong_way_driving\
                + any_drunk_drivers + num_drunk_participants + pedestrian_related'

model = smf.mnlogit(formula, data=df_balanced)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 1.017238
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:        severity_encode   No. Observations:                59080
Model:                        MNLogit   Df Residuals:                    59064
Method:                           MLE   Df Model:                           14
Date:                Sun, 20 Apr 2025   Pseudo R-squ.:                 0.02582
Time:                        06:04:54   Log-Likelihood:                -60098.
converged:                       True   LL-Null:                       -61691.
Covariance Type:            nonrobust   LLR p-value:                     0.000
     severity_encode=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0519      0.017     -3.047      0.002      -0.085      

`'is_toll'` shows a positive and statistically significant effect for both medium-severity crashes (coef = 0.0619, p < 0.001) and severe crashes (coef = 0.1602, p < 0.001). This means that on toll roads, the likelihood of more severe outcomes compared to minor ones is significantly higher, which **supports Hypothesis 2.1** in this model specification.

Next, environmental factors, vehicle technical characteristics, and participant behavior should be combined into a single model, which brings the analysis closer to a more comprehensive assessment of the `'is_toll'` effect.

In [137]:
formula = 'severity_encode ~ is_toll\
          + num_drunk_participants + guilty_drivers_avg_experience\
          + share_guilty_vehicles\
          + lighting_cat + road_object_type_cat\
          + speeding + signal_violation + pedestrian_related + wrong_way_driving\
          + num_front_wheel_drive_vehicles + avg_vehicles_age\
          + dor_z_cat + DTPV_cat + OBJ_DTP_cat + factor_cat'


model = smf.mnlogit(formula, data=df_balanced)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 1.006955
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:        severity_encode   No. Observations:                59080
Model:                        MNLogit   Df Residuals:                    59046
Method:                           MLE   Df Model:                           32
Date:                Sun, 20 Apr 2025   Pseudo R-squ.:                 0.03567
Time:                        06:04:55   Log-Likelihood:                -59491.
converged:                       True   LL-Null:                       -61691.
Covariance Type:            nonrobust   LLR p-value:                     0.000
             severity_encode=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         39.1559      2.767     14.151     

The coefficient for `'is_toll'` is positive and statistically significant for both medium-severity crashes (coef = 0.1272, p < 0.001) and severe crashes (coef = 0.0728, p = 0.012).  
Thus, the model confidently **confirms Hypothesis 2.1**: the likelihood of both medium and severe crashes is higher on toll roads compared to minor ones when controlling for the full set of factors.

### 2. sample weights

In [143]:
sample_weights = compute_sample_weight(class_weight='balanced', y=df_hypoth['is_toll'])
df_weighted = df_hypoth.loc[df_hypoth.index.repeat(sample_weights.round().astype(int))]

X = df_weighted[[
    'is_toll', 'RTA_vehicle_number', 'RTA_participants_number',
    'num_drunk_participants', 'guilty_drivers_avg_experience',
    'drivers_avg_experience', 'share_guilty_vehicles',
    'lighting_cat', 'road_object_type_cat', 'speeding', 'signal_violation',
    'pedestrian_related', 'wrong_way_driving', 'num_front_wheel_drive_vehicles',
    'avg_vehicles_age', 'dor_z_cat', 'DTPV_cat', 'OBJ_DTP_cat', 'factor_cat'
]]

X = sm.add_constant(X)

y = df_weighted['severity_encode']

# model = sm.MNLogit(y, X)
# result = model.fit(weights=sample_weights)

model = sm.MNLogit(y, X)
result = model.fit()

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.987080
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:        severity_encode   No. Observations:               617372
Model:                        MNLogit   Df Residuals:                   617332
Method:                           MLE   Df Model:                           38
Date:                Sun, 20 Apr 2025   Pseudo R-squ.:                 0.04488
Time:                        06:25:53   Log-Likelihood:            -6.0940e+05
converged:                       True   LL-Null:                   -6.3803e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
             severity_encode=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             28.8096      0.816     35.298     

The coefficient for `'is_toll'` is positive and highly significant for both medium-severity crashes (coef = 0.2600, p < 0.001) and severe crashes (coef = 0.2564, p < 0.001), providing strong support for Hypothesis 2.1.  
The significance of `is_toll` remains even after controlling for more than 15 different predictors, which confirms the robustness of the toll road effect.

_To test the robustness of the results, a series of models was estimated with the gradual inclusion of variables (on a balanced subsample). The weighted model was built using the broadest specification, reflecting the full range of available factors, which allowed for the most comprehensive view of the toll road’s impact on crash severity._